In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cyberbully-detection-dataset/cb_multi_labeled_balanced.csv
/kaggle/input/cyberbullying-classification/cyberbullying_tweets.csv


In [2]:
df=pd.read_csv('/kaggle/input/cyberbullying-classification/cyberbullying_tweets.csv')

In [3]:
df['cyberbullying_type'].value_counts()

cyberbullying_type
religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: count, dtype: int64

In [4]:
df.sample(10)

,tweet_text,cyberbullying_type
9847,@callmemrcarter: I hope Kat gets Ebola and the...,gender
25009,RT @Coopster04: .@DKParagon As a mom of daught...,other_cyberbullying
15488,"ppl make black, holocaust, rape, and gay jokes...",gender
21854,@pNibbler @AlterNet @MaxBlumenthal In every Ch...,religion
12896,prison rape isn't funny. prison rape jokes are...,gender
25682,@JackWilshere @piersmorgan What an idiot @joey...,other_cyberbullying
14123,RT @perydwyn: @MT8_9 @FactsVsOpinion @VettedVi...,gender
13119,gay jokes rape jokes dead baby jokes racial jo...,gender
2511,YEAH VICTORIA #MKR,not_cyberbullying
18266,Nothing surprising Continue Mashaallah For Med...,religion


In [ ]:
df.columns

In [5]:
df2=pd.read_csv('/kaggle/input/cyberbully-detection-dataset/cb_multi_labeled_balanced.csv')
df2.sample(10)

,text,label
92363,@RevivalofMan Fuck these Hindu mother fuckers....,religion
52792,There's still plenty of time to join The Great...,not_cyberbullying
56858,"@sensyu_renai Ummm, anybody h (as a link to th...",not_cyberbullying
58222,"After being taken from her family in 1970, the...",not_cyberbullying
22931,"Listen, Muslims only see us as the enemy, ever...",religion
79462,@Kevin_Maguire Can you bring up Zahawi on gmtv...,not_cyberbullying
34931,[UPDATED JANUARY 24] Dragonflight: 10.0.5 Cont...,not_cyberbullying
59670,@Izzie_Moss @francesweetman There are 17 rape ...,gender/sexual
76115,@NFL @SnoopDogg @EliManning Peyton Manning (QB...,not_cyberbullying
26043,#eth #btc will give you peace at night while w...,not_cyberbullying


In [6]:
filtered_df2 = df2[df2['label'] == 'not_cyberbullying']
filtered_df2.rename(columns={'text': 'tweet_text'}, inplace=True)
filtered_df2.rename(columns={'label': 'cyberbullying_type'}, inplace=True)
filtered_df2.columns
# Concatenate filtered_df2 with df
#combined_df = pd.concat([filtered_df2,df], ignore_index=True)

/tmp/ipykernel_269/2434627164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df2.rename(columns={'text': 'tweet_text'}, inplace=True)
/tmp/ipykernel_269/2434627164.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df2.rename(columns={'label': 'cyberbullying_type'}, inplace=True)


Index(['tweet_text', 'cyberbullying_type'], dtype='object')

In [7]:
combined_df = pd.concat([filtered_df2,df], ignore_index=True)
combined_df.shape

(97692, 2)

In [8]:
combined_df['cyberbullying_type'].value_counts()


cyberbullying_type
not_cyberbullying      57945
religion                7998
age                     7992
gender                  7973
ethnicity               7961
other_cyberbullying     7823
Name: count, dtype: int64

In [ ]:
combined_df.sample(50)

In [9]:
combined_df['label'] = np.where(combined_df['cyberbullying_type'] == 'not_cyberbullying', 0, 1)

In [10]:
combined_df=combined_df.drop(['cyberbullying_type'],axis=1)

In [14]:


# Separate the DataFrame into two based on the labels
df_label_0 = combined_df[combined_df['label'] == 0]
df_label_1 = combined_df[combined_df['label'] == 1]

# Sample 50% of each subset
df_label_0_sampled = df_label_0.sample(frac=0.4)
df_label_1_sampled = df_label_1.sample(frac=0.5)

# Concatenate the sampled DataFrames back together
df_sampled = pd.concat([df_label_0_sampled, df_label_1_sampled])

# Optionally, save the resulting DataFrame to a new CSV file
df_sampled.head(10)


,tweet_text,label
34279,"was about to make dinner, but took a break and...",0
42814,Same except I was 20 at the beginning and I’m ...,0
8499,@johngunden Good luck! I'm praying for you.,0
5594,@MonicaLennon7 Devolution working as it was wr...,0
42646,@_madisonmariela I´m quite good at statistics.,0
42778,@TimMaxwell22 Would he be eligible for more if...,0
7474,@kimballzzzzz Was at the second game in Anahei...,0
38592,@thecatlady3132 Yup..!! Great thanks..!! Just ...,0
2071,Thinking of Estinien showing see how to gently...,0
3783,Registered Nurse Functional Assessor\nDo good....,0


In [15]:
df_sampled['label'].value_counts()

label
0    23178
1    19874
Name: count, dtype: int64

In [16]:
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
def process(text):
    text=re.sub(text_cleaning_re,' ',str(text).lower()).strip()
    tokens=[]
    for i in text.split():
        if i not in stop_words:
            tokens.append(stemmer.stem(i))
    return ' '.join(tokens)
df_sampled['tweet_text']=df_sampled['tweet_text'].map(lambda x:process(x))
df_sampled.head()

,tweet_text,label
34279,make dinner took break sat end eat 1000 cal wo...,0
42814,except 20 begin turn 23 year feel like still 19,0
8499,good luck pray,0
5594,devolut work written,0
42646,quit good statist,0


In [ ]:
!pip install torch torchvision transformers pandas scikit-learn


In [20]:
# Adjust path accordingly
texts = df_sampled['tweet_text'].tolist()  # Assuming the text column is named 'text'
labels = df_sampled['label'].tolist()  # Assuming the label column is named 'label'


In [21]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def generate_embeddings(texts, batch_size=32):
    model.eval()
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        with torch.no_grad():
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=128)
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)
            embeddings.append(batch_embeddings)
    embeddings = torch.cat(embeddings, dim=0)
    return embeddings

# Assuming df['text'] contains your text data
embeddings = generate_embeddings(texts, batch_size=16) 

In [22]:
from sklearn.model_selection import train_test_split

# Assuming `embeddings` is your array of BERT embeddings and `labels` is your array of labels
X_train, X_temp, y_train, y_temp = train_test_split(embeddings, labels, test_size=0.25, stratify=labels, random_state=42)



In [25]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Compute cosine similarity between embeddings
cos_sim = cosine_similarity(X_train)

# Define a threshold for connecting nodes (tweets)
threshold = 0.9  # This is arbitrary and should be selected based on your specific needs

# Create edges based on the threshold
edges = np.where(cos_sim > threshold)

# Prepare for PyTorch Geometric
edge_index = torch.tensor(edges, dtype=torch.long)


In [27]:
!pip install torch_geometric
from torch_geometric.data import Data

# Convert labels to tensor
labels_tensor = torch.tensor(y_train, dtype=torch.long)

# Create a graph data object
train_data = Data(x=X_train, edge_index=edge_index, y=labels_tensor)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 614.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.1 MB/s eta 0:00:0000:0100:01


In [28]:
cos_sim = cosine_similarity(X_temp)

# Define a threshold for connecting nodes (tweets)
threshold = 0.9  # This is arbitrary and should be selected based on your specific needs

# Create edges based on the threshold
edges = np.where(cos_sim > threshold)

# Prepare for PyTorch Geometric
edge_index = torch.tensor(edges, dtype=torch.long)
labels_tensor = torch.tensor(y_temp, dtype=torch.long)

# Create a graph data object
test_data = Data(x=X_temp, edge_index=edge_index, y=labels_tensor)

In [35]:
num_features = train_data.num_features
type(train_data.x)

Data(x=[32289, 768], edge_index=[2, 349345], y=[32289])

In [77]:
type(train_data.x)

torch.Tensor

In [78]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.data import DataLoader

class GCN(torch.nn.Module):
    def __init__(self,num_features,hidden_channels,hidden_channels2,out_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features,hidden_channels)
        self.conv2 = GCNConv(hidden_channels,hidden_channels)
        self.conv3= GCNConv(hidden_channels,hidden_channels)
        self.fc1=torch.nn.Linear(hidden_channels,hidden_channels2)
        self.fc2=torch.nn.Linear(hidden_channels2,out_classes)# Assuming you know the number of classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))

        # Flatten for fully connected layers
        x = F.dropout(x, p=0.5, training=self.training)
        
        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

model = GCN(num_features,64,32,1)
model



GCN(
  (conv1): GCNConv(768, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)

In [79]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [80]:
from torch_geometric.data import Dataset

class CustomGraphDataset(Dataset):
    def __init__(self, data, transform=None, pre_transform=None):
        self.data = data
        super(CustomGraphDataset, self).__init__(transform, pre_transform)

    @property
    def raw_file_names(self):
        # Not used in this example
        return []

    @property
    def processed_file_names(self):
        # Not used in this example
        return []

    def download(self):
        # Not used in this example
        pass

    def process(self):
        # Not used in this example
        pass

    def len(self):
        # For a single graph, return 1
        return 1

    def get(self, idx):
        # Return the single graph
        return self.data


In [81]:
train_dataset = CustomGraphDataset(data=train_data)
test_dataset=CustomGraphDataset(data=test_data)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

Processing...
Done!
Processing...
Done!


In [82]:
batch=next(iter(loader))
x,y=batch.x,batch.y
batch

DataBatch(x=[32289, 768], edge_index=[2, 349345], y=[32289], batch=[32289], ptr=[2])

In [84]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [68]:
from sklearn.metrics import accuracy_score, f1_score


def train(epoch):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(torch.reshape(out, (-1,)), data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test(epoch):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(torch.reshape(out, (-1,)), data.y.float())
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())

    # Calculate Metrics
    accuracy, f1 = metrics(all_preds, all_labels)

    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    acc = accuracy_score(preds, gts)
    f1 = f1_score(preds, gts)
    return acc, f1

AttributeError: 'DataLoader' object has no attribute 'to'

In [87]:
model.to(device)
model.train()
for epoch in range(500):  # Number of epochs
    total_loss = 0
    for batch in train_loader:
        batch.to(device)  # Move batch to the appropriate device
        optimizer.zero_grad()  # Clear gradients
        out = model(batch)  # Forward pass
        loss = criterion(out.squeeze(), batch.y.float())  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update model parameters
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}: Loss: {avg_loss:.4f}')

Epoch 1: Loss: 0.6418
Epoch 2: Loss: 0.6304
Epoch 3: Loss: 0.6177
Epoch 4: Loss: 0.6035
Epoch 5: Loss: 0.5896
Epoch 6: Loss: 0.5741
Epoch 7: Loss: 0.5573
Epoch 8: Loss: 0.5426
Epoch 9: Loss: 0.5250
Epoch 10: Loss: 0.5100
Epoch 11: Loss: 0.4956
Epoch 12: Loss: 0.4824
Epoch 13: Loss: 0.4709
Epoch 14: Loss: 0.4606
Epoch 15: Loss: 0.4527
Epoch 16: Loss: 0.4463
Epoch 17: Loss: 0.4368
Epoch 18: Loss: 0.4320
Epoch 19: Loss: 0.4244
Epoch 20: Loss: 0.4200
Epoch 21: Loss: 0.4140
Epoch 22: Loss: 0.4098
Epoch 23: Loss: 0.4063
Epoch 24: Loss: 0.3997
Epoch 25: Loss: 0.3951
Epoch 26: Loss: 0.3899
Epoch 27: Loss: 0.3870
Epoch 28: Loss: 0.3836
Epoch 29: Loss: 0.3794
Epoch 30: Loss: 0.3754
Epoch 31: Loss: 0.3756
Epoch 32: Loss: 0.3725
Epoch 33: Loss: 0.3696
Epoch 34: Loss: 0.3676
Epoch 35: Loss: 0.3666
Epoch 36: Loss: 0.3640
Epoch 37: Loss: 0.3614
Epoch 38: Loss: 0.3597
Epoch 39: Loss: 0.3571
Epoch 40: Loss: 0.3542
Epoch 41: Loss: 0.3535
Epoch 42: Loss: 0.3525
Epoch 43: Loss: 0.3494
Epoch 44: Loss: 0.34

In [95]:
accuracy_score

AttributeError: 'Tensor' object has no attribute 'append'

In [90]:
from sklearn.metrics import accuracy_score, f1_score

In [96]:
from sklearn.metrics import accuracy_score, f1_score
import torch

def metrics(all_preds, all_labels):
    # Concatenate all batches
    preds = torch.cat(all_preds, dim=0)
    labels = torch.cat(all_labels, dim=0)

    # Convert predictions to binary by applying a threshold (0.5 for sigmoid output)
    binary_preds = torch.round(torch.sigmoid(preds))

    # Calculate metrics
    accuracy = accuracy_score(labels.cpu(), binary_preds.cpu())
    f1 = f1_score(labels.cpu(), binary_preds.cpu())

    return accuracy, f1

def test(test_loader, model, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():  # No gradients needed for evaluation
        for data in test_loader:
            data = data.to(device)
            out = model(data)
            # Ensure the output and labels have the correct shape
            loss = criterion(out.view(-1), data.y.float().to(device))
            total_loss += loss.item() * data.num_graphs
            # Collect predictions and labels for metric calculation
            all_preds.append(out.view(-1))
            all_labels.append(data.y.float().to(device))

    # Calculate Metrics
    accuracy, f1 = metrics(all_preds, all_labels)

    avg_loss = total_loss / len(test_loader.dataset)
    return avg_loss, accuracy, f1
test(test_loader,model,criterion,device)

(0.5803821682929993, 0.849391433615163, 0.8342705244862488)